In [3]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

from twitter_scraper import settings
# from twitter_scraper.text.tweets import detect_language
from twitter_scraper.text.tweets import get_stemmed_text
from twitter_scraper.text.tweets import clean_twitter_text
from twitter_scraper.clean.users import USER_DTYPE
from twitter_scraper.clean.tweets import TWEET_DTYPE


users_df = pd.read_csv(settings.USERS_CSV, dtype=USER_DTYPE)
tweets_df = pd.read_csv(settings.TWEETS_CSV, dtype=TWEET_DTYPE)
retweets_df = pd.read_csv(settings.EDGES_RETWEETS_CSV)
retweets_df = retweets_df.loc[retweets_df['source'] != retweets_df['target']]

user_idx = users_df[['user_id', 'screen_name']].set_index('user_id').to_dict()['screen_name']
retweets_df['source'] = retweets_df['source'].transform(lambda x: user_idx[x])
retweets_df['target'] = retweets_df['target'].transform(lambda x: user_idx[x])
# retweets_df['full_text'] = retweets_df['full_text'].transform(clean_twitter_text)
# retweets_df['langid'] = retweets_df['full_text'].transform(detect_language)

In [4]:
# retweets_df['stemmed_text'] = retweets_df['full_text'].transform(get_stemmed_text)
retweets_df

,source,target,rt_tweet_id,full_text,og_tweet_id,rt_time_elapsed_sec
0,sdewes_centre,TihomirTomic,1504789813433671682,RT @sdewes_centre: SPECIAL ISSUES :: What is t...,1481965822100729856,5441663.0
2,HNS_CFF,LukaVeso,1575092693482414080,RT @HNS_CFF: 🇭🇷 #Croatia on 🔝 seems just fine ...,1575027254232764416,NaN
3,gnkdinamo,LukaVeso,1567224091236614149,RT @gnkdinamo: FT 90'+8 Dinamo - Chelsea 1:0 (...,1567221760524795904,556.0
4,HNS_CFF,LukaVeso,1562569249687605249,RT @HNS_CFF: Čestitamo! @gnkdinamo branit će h...,1562569135804264448,NaN
5,natasazecevic,susacp11,1526577980746649610,RT @natasazecevic: Rastavio ga https://t.co/UE...,1526468312099586048,NaN
...,...,...,...,...,...,...
53509,hkresic,DNEVNIKhr,1582314780836384769,RT @hkresic: Davor Mayer odgovara Zekanoviću (...,1582314230149041920,NaN
53510,BuljanNovaTV,DNEVNIKhr,1582314762259828736,RT @BuljanNovaTV: Hrvoje Zekanović tvrdi - 5 M...,1582311143002890240,NaN
53511,hkresic,DNEVNIKhr,1582314743087316993,RT @hkresic: Nakon što je izazvao incident na ...,1582312816291491840,NaN
53512,BuljanNovaTV,DNEVNIKhr,1582299166817910784,RT @BuljanNovaTV: Počela sjednica Antikorupcij...,1582299038384201728,30.0


In [ ]:
# retweet_tweet_ids = retweets_df.og_tweet_id.value_counts().index
retweets_df.og_tweet_id.value_counts()

In [ ]:
tweets_df.langid.value_counts()[:15].plot.bar(rot=0)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(24, 8))

source_retweets_df = pd.DataFrame(retweets_df.source.value_counts())[:15].join(users_df[['user_id', 'screen_name']].set_index('user_id')).sort_values('source', ascending=False)
source_retweets_df.plot.bar(x='screen_name', y='source', rot=30, title='Most retweeted sources', ax=axs[0])

target_retweets_df = pd.DataFrame(retweets_df.target.value_counts())[:15].join(users_df[['user_id', 'screen_name']].set_index('user_id')).sort_values('target', ascending=False)
target_retweets_df.plot.bar(x='screen_name', y='target', rot=30, title='Most retweeting targets', ax=axs[1])
plt.show()

In [ ]:
tweets_df[tweets_df['id'] == str(1501227789437255680)]

In [ ]:
retweets_df.loc[retweets_df['source'].isin(source_retweets_df.index) | retweets_df['target'].isin(target_retweets_df.index)]

In [ ]:
fig, ax = plt.subplots(figsize=(18, 15))


G = nx.from_pandas_edgelist(retweets_df.loc[retweets_df['source'].isin(source_retweets_df.index) | retweets_df['target'].isin(target_retweets_df.index)], edge_attr='rt_time_elapsed_sec')
nx.draw_spring(G, node_size=20, ax=ax, labels=user_idx)

In [ ]:
G = nx.from_pandas_edgelist(retweets_df, create_using=nx.DiGraph)
dg_centrality = nx.degree_centrality(G)
dg_centrality = [{'user_id': key, 'user_name': users_df.loc[users_df['user_id'] == key].screen_name.values[0], 'centrality': value} for key, value in dg_centrality.items()]
sorted_dg_centrailty = sorted(dg_centrality, key=lambda x: x['centrality'], reverse=True)
best_dg_centrality = sorted_dg_centrailty[:10]
best_dg_centrality

In [ ]:
from twitter_scraper.utils import fileio
from twitter_scraper.text.tweets import get_stemmed_text

best_dg_centrality_user_ids = [item['user_id'] for item in best_dg_centrality]

def get_central_user_tweets():
    user_tweet_records = tweets_df[tweets_df['user_id'].isin(best_dg_centrality_user_ids)][['user_id', 'full_text']].to_dict(orient='records')
    central_user_tweets = {}
    for item in user_tweet_records:
        user_id = item['user_id']
        if user_id in central_user_tweets:
            central_user_tweets[user_id].append(item['full_text'])
        else:
            central_user_tweets[user_id] = [item['full_text']]
    return central_user_tweets


central_user_tweets = get_central_user_tweets()
most_occuring_keywords = {}
for user_id in best_dg_centrality_user_ids:
    word_count = most_occuring_keywords.get(user_idx[user_id], {})
    for text in central_user_tweets[user_id]:
        text = get_stemmed_text(text)
        for word in text:
            if word in word_count:
                word_count[word] += 1
            else:
                word_count[word] = 1
    most_occuring_keywords[user_idx[user_id]] = word_count
# fileio.write_content('most_occuring_keywords.json', most_occuring_keywords, 'json')
print(most_occuring_keywords[user_idx[user_id]])

In [ ]:
tweets_df[tweets_df['user_id'].isin(best_dg_centrality_user_ids)][['user_id', 'full_text']].to_dict(orient='records')

In [ ]:
import tweepy

conn_details = settings.connections['andhrelja']
auth = tweepy.OAuthHandler(conn_details['consumer_key'], conn_details['consumer_secret'])
auth.set_access_token(conn_details['access_key'], conn_details['access_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
retweets = api.get_retweets(1555473887336472576)

In [ ]:
len(retweets)

In [ ]:
pd.DataFrame(api.supported_languages()).sort_values('code').reset_index(drop=True)